# Jupyter Notebook best practices for working with SQL

During the analysis phase it can be cumbersome to manage SQL connections and passing string with SQL queries around, e.g. by using `sqlalchemy`. This notebook showcases the application of the `ipython-sql` extension for Jupyter notebooks, which greatly simplifies using SQL and `pandas` during an analysis.

## Setup

First, let's define the database connection url in an environment variable called `DATABASE_URL`. This connection url will be used by the [ipython-sql](https://pypi.org/project/ipython-sql/) extension.

In [ ]:
%env DATABASE_URL=sqlite:///data/ts.db

This repository already contains a simple database file called ts.db. This file has been created by the cell below. You can simply skip the next cell to use that file or re-run it to overwrite the existing file with new random data.

In [ ]:
import os
import sqlalchemy
engine = sqlalchemy.create_engine(os.environ['DATABASE_URL'])

import pandas as pd
import pandas.util.testing as pdt
pdt.N = 12
pdt.K = 6
data = pdt.makeTimeDataFrame(freq='MS')

data.to_sql('data', index=True, index_label='ts', if_exists='replace', con=engine)

Let's see which columns we have available.

In [ ]:
data.columns

## SQL magic

Before we can use the `%sql` cell magics we have to load the ipython SQL extension.

In [ ]:
# %pip install ipython-sql

In [ ]:
%load_ext sql

### Queries

A single line SQL query can be executed by prefixing the query with `%sql`.

In [ ]:
%sql SELECT ts, A FROM data WHERE A < 0

The result of such a query can be directly assigned to a python variable. Note however, that the result is not a `pandas.DataFrame` but a `sql.run.ResultSet`.

In [ ]:
negative_bs = %sql SELECT B FROM data WHERE B < 0

In [ ]:
type(negative_bs)

One may want to spread a query over multiple lines to increase readability when the query becomes more complex. This can be done by using `%%sql` instead of just `%sql`.

In [ ]:
%%sql
WITH summer_es AS (
SELECT
    E,
    CAST(strftime('%m', ts) AS INTEGER) AS month
FROM
    data
WHERE
    month in (6,7,8)
)
SELECT 
    * 
FROM
    summer_es
WHERE
    E > 0
ORDER BY
    E DESC

The result of a multi-line query can be assigned to a variable as well. The syntax is a bit different though. Note the `<<` operator.

In [ ]:
%%sql positive_summer_es <<
WITH summer_es AS (
SELECT
    CAST(strftime('%m', ts) AS INTEGER) AS month,
    E
FROM
    data
WHERE
    month in (6,7,8)
)
SELECT 
    * 
FROM
    summer_es
WHERE
    E > 0
ORDER BY
    E DESC

### Ploting

A `ResultSet` offers some convenience functions for plotting.

In [ ]:
%matplotlib inline

In [ ]:
positive_summer_es.bar()

### Exporting

It is also possible to export or convert the query result.

#### file

A `ResultSet` can be written directly to a csv file.

In [ ]:
positive_summer_es.csv(filename='positive_summer_es.csv')

#### pandas.DataFrame

Converting to a `pandas.DataFrame` is equally supported.

In [ ]:
pos_summer_sales = positive_summer_es.DataFrame()

In [ ]:
pos_summer_sales.dtypes

#### database table

Finally, a `pandas.DataFrame` can be persisted to a database table without much effort. The resulting table in the database will have the same name as the dataframe variable.

In [ ]:
%sql --persist pos_summer_sales

Query the newly created table.

In [ ]:
%sql SELECT * FROM pos_summer_sales